# Algerian Forest Fires: How different weather conditions induce forest fires

# Introduction

- Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
 
    
- Clearly state the question you will try to answer with your project
    
How do relative humidity, wind speed, rain, and temperature predict whether a forest fire occurred in the Bejaia region?
    
- Identify and describe the dataset that will be used to answer the question
 
We will be using the Algerian Forest Fires Data Set, which primarily contains numerical data. Using the numerical values for the predictors: relative humidity, wind speed, rain, and temperature, we can analyze how all those predictors, when combined, predict whether or not a fire forest fire occurred in the Bejaia region. 


# Preliminary Exploratory Data Analysis

- Demonstrate that the dataset can be read from the web into R 
- Clean and wrangle your data into a tidy format
- **Using only training data**, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
- **Using only training data**, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.


# Methods

- Explain how you will conduct either your data analysis and which variables/columns you will use. 
    
Using K-nearest neighbours classification, we will use the columns: RH (relative humidity), Ws (wind speed), Rain, and Temperature as predictors to determine if a forest fire occurred in Bejaia and Sidi-Bel Abbes regions.  

- Describe at least one way that you will visualize the results
 
 We can visualize the results using a scatter plot. 


# Expected Outcomes and Significance

- What do you expect to find?
    - absences: inverse relationship
    - extra educational support: more support = better grades?
    - study time: limited rate of return
    - internet access at home: internet access = better grades
    
 When combined, lower absences, increased extra educational support, increased study time, and having access to internet at home should predict a final grade of at least 10. **DECIDE!!!!**
    
- What impact could such findings have?
    
- What future questions could this lead to?
